In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.0.8'

# 初识神经网络

本笔记本包含第2章[深度学习与Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff)第1部分中的代码示例。请注意，原文内容更多，特别是更多解释和图片：在本笔记本中，您只能找到源代码和相关评论。

----

现在我们来看一个神经网络的第一个具体例子，它使用Python库Keras学习分类手写数字。除非你已经有Keras或类似的函数库的使用经验，否则你不能马上完全理解关于这个第一个例子。你可能还没有安装Keras。别担心，那很好。在下一章中，我们将回顾我们示例中的每个元素并详细解释它们。所以，不要担心，如果某些步骤看起来是任意的或者对你来说看起来像是魔法！我们必须从某个地方开始。
我们在这里试图解决的问题是将手写数字（28像素×28像素）的灰度图像分类为10个类别（0到9）。我们将使用的数据集是MNIST数据集，这是机器学习社区中的一个经典数据集，它几乎与该领域本身存在一样长的时间，并且已经进行了非常深入的研究。这是一套60,000张训练图像，外加10,000张测试图像，由美国国家标准与技术研究院（NIST）于20世纪80年代组装而成。你可以将“解决”MNIST想象成深度学习的“Hello World” - 这就是你如何验证你的算法是否按预期工作。当你成为一名机器学习实践者时，你会看到MNIST一次又一次地在科学论文，博客文章等等中出现。

MNIST数据集在Keras中预先加载，其形式为一组四个Numpy数组：

In [2]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

`train_images`并`train_labels`形成“训练集”，即模型将学习的数据。该模型将被由`test_images`和`test_labels`组成的“测试组”测试，。我们的图像被编码为Numpy数组，标签只是一个数字数组，范围从0到9.图像和标签之间存在一一对应的关系。
我们来看看训练数据：

In [3]:
train_images.shape

(60000, 28, 28)

In [4]:
len(train_labels)

60000

In [5]:
train_labels

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

我们来看看测试数据：

In [6]:
test_images.shape

(10000, 28, 28)

In [7]:
len(test_labels)

10000

In [8]:
test_labels

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

我们的工作流程如下：首先，我们将向我们的神经网络提供训练数据，`train_images`以及`train_labels`。网络将学习将图像和标签关联起来。最后，我们将要求网络产生预测`test_images`，并且我们将验证这些预测是否与标签匹配`test_labels`。

让我们建立我们的网络 - 再一次记住：你不一定能完全理解这个例子。

In [9]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

神经网络的核心构建块是“层”，这是一个数据处理模块，您可以将其视为数据的“过滤器”。一些数据进入，并以更有用的形式出现。准确地说，图层从提供给它们的数据中提取新的数据表示(即通过对数据处理提取有用信息，译者注） - 希望对于手头问题更有意义的表示。大多数深度学习实际上是将简单的层连接在一起，这些层将实现一种渐进的“数据蒸馏”形式。深度学习模式就像是一个数据处理筛选器，由一系列越来越精细的数据过滤器 - “层”组成。

这里我们的网络由一系列`Dense`两层紧密连接（也称为“完全连接”）的神经层组成。第二层（也是最后一层）是10路“softmax”层，这意味着它将返回10个概率分数的数组（总计为1）。每个分数将是当前数字图像属于我们的10位数类别之一的概率。

为了让我们的网络准备好接受培训，我们需要选择另外三件事，作为“编译”步骤的一部分：

- 损失函数(loss function)：网络将如何衡量其在训练数据上的工作性能，以及如何能够在正确的方向上引导自己。
- 优化器(optimizer)：这是网络根据输入数据计算得来的误差值（通过损失函数计算）自行更新的机制。
- 指标(metrics)在培训和测试期间进行监控。在这里，我们只关心准确度（被正确分类的图像部分）。
损失函数和优化器的确切目的将在接下来的两章中阐明。

In [10]:
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

在培训之前，我们将通过将数据重新整理为网络预期的形状来对数据进行预处理，并对其进行缩放，以使所有值都处于`[0, 1]`间隔中。我们的训练图像存储在形状为`(60000, 28, 28)`三维数组中，其数据为处于`[0,255]`之间的`unit8`类型数据。我们将其转换为值为0到1之间的`float32`数据。

In [11]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

我们还需要对标签进行分类编码，这是我们将在第3章中解释的一个步骤：

In [12]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

我们现在准备好培训我们的网络，在Keras中通过调用`fit`将训练模型。

In [13]:
network.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
60000/60000 [==============================] - 2s - loss: 0.2577 - acc: 0.9245     
Epoch 2/5
60000/60000 [==============================] - 1s - loss: 0.1042 - acc: 0.9690     
Epoch 3/5
60000/60000 [==============================] - 1s - loss: 0.0687 - acc: 0.9793     
Epoch 4/5
60000/60000 [==============================] - 1s - loss: 0.0508 - acc: 0.9848     
Epoch 5/5
60000/60000 [==============================] - 1s - loss: 0.0382 - acc: 0.9890     


训练期间显示两个数据输出：网络在训练数据上的“损失”以及网络在训练数据上的准确性。

我们很快就达到了训练数据的0.989（即98.9％）的准确度。现在让我们来检查一下我们的模型在测试集上的表现，当然也很好：

In [14]:
test_loss, test_acc = network.evaluate(test_images, test_labels)

 9536/10000 [===========================>..] - ETA: 0s

In [15]:
print('test_acc:', test_acc)

test_acc: 0.9777


我们的测试集精度为97.8％ - 这比训练集精度要低很多。训练数据集上的准确性和测试数据集上的准确性之间的这种差距是“过度拟合”的一个例子，机器学习模型在新数据上的表现往往比在训练数据上更差。过度拟合将成为第3章的核心话题。

这就是我们第一个例子的结束 - 您刚刚看到我们如何构建并训练一个神经网络来对少于20行的Python代码中的手写数字进行分类。在下一章中，我们将详细介绍我们刚才预览的每一个动态片段，并阐明幕后真正发生的事情。您将学习进入网络的数据存储对象“张量”，张量操作，层构成以及梯度下降，这使得我们的网络可以从其训练示例中学习。